#IR ASSIGNMENT 2

In [ ]:
import pandas as pd
df = pd.read_csv("A2_Data.csv")

In [ ]:
df.head(10)

,id,Image,Review Text
0,3452,['https://images-na.ssl-images-amazon.com/imag...,Loving these vintage springs on my vintage str...
1,1205,['https://images-na.ssl-images-amazon.com/imag...,Works great as a guitar bench mat. Not rugged ...
2,1708,['https://images-na.ssl-images-amazon.com/imag...,We use these for everything from our acoustic ...
3,2078,['https://images-na.ssl-images-amazon.com/imag...,Great price and good quality. It didn't quite...
4,801,['https://images-na.ssl-images-amazon.com/imag...,I bought this bass to split time as my primary...
5,126,['https://images-na.ssl-images-amazon.com/imag...,"it's more on toy side than on instrument side,..."
6,1329,['https://images-na.ssl-images-amazon.com/imag...,Absolute BEST guitar hangers on the market... ...
7,325,['https://images-na.ssl-images-amazon.com/imag...,"Great nylon strings, just as expected. They wo..."
8,245,['https://images-na.ssl-images-amazon.com/imag...,I bought this stand for church because I been ...
9,1714,['https://images-na.ssl-images-amazon.com/imag...,Awesome stand!\n\nTip: The bottom part that su...


In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('A2_Data.csv')

# Create an empty DataFrame to store the split image links
new_rows = []

# Iterate through each row of the original DataFrame
for index, row in df.iterrows():
    image_data = row['Image']
    # Split the image links
    image_data = image_data.strip(' []')
    # Split the image URLs and metadata on comma
    image_links = image_data.split(',')
    # Create a new row for each image link
    for link in image_links:
        new_row = {
            'id': row['id'],
            'image_link': link.strip().strip("'"),  # Remove leading/trailing spaces
            'review_text': row['Review Text']
        }
        new_rows.append(new_row)

# Create a new DataFrame from the list of new rows
new_df = pd.DataFrame(new_rows)
new_df
# Save the new DataFrame to a new CSV file
new_df.to_csv('split_image_links.csv', index=False)

In [ ]:
import pandas as pd
import requests
import os
from PIL import Image
from io import BytesIO

# Read the CSV file into a DataFrame
df = pd.read_csv('split_image_links.csv')

# Create a directory named "images" if it doesn't exist
if not os.path.exists('Images'):
    os.makedirs('Images')

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Extract the image URLs and metadata from the 'Image' column
    image_data = row['image_link']
    metadata = row['review_text']
    # Get the id from the 'id' column
    id = row['id']
    # Remove leading and trailing whitespace and single quote

    # Fetch the image from the URL
    response = requests.get(image_data)
    if response.status_code == 200:
        # Open the image using PIL
        image = Image.open(BytesIO(response.content))
        # Save the image to the "images" directory with corresponding metadata
        filename = f"{index+1}_{id}_image.jpg"
        image.save(os.path.join('Images', filename))
        print(f"Image {filename} saved successfully with metadata: {id}-{index+1}")
    else:
        print(f"Failed to fetch image {index}_{id} ")


Image 1_3452_image.jpg saved successfully with metadata: 3452-1
Image 2_1205_image.jpg saved successfully with metadata: 1205-2
Image 3_1205_image.jpg saved successfully with metadata: 1205-3
Image 4_1205_image.jpg saved successfully with metadata: 1205-4
Image 5_1708_image.jpg saved successfully with metadata: 1708-5
Image 6_2078_image.jpg saved successfully with metadata: 2078-6
Image 7_801_image.jpg saved successfully with metadata: 801-7
Image 8_126_image.jpg saved successfully with metadata: 126-8
Image 9_126_image.jpg saved successfully with metadata: 126-9
Image 10_126_image.jpg saved successfully with metadata: 126-10
Image 11_126_image.jpg saved successfully with metadata: 126-11
Image 12_1329_image.jpg saved successfully with metadata: 1329-12
Image 13_325_image.jpg saved successfully with metadata: 325-13
Image 14_325_image.jpg saved successfully with metadata: 325-14
Image 15_325_image.jpg saved successfully with metadata: 325-15
Image 16_325_image.jpg saved successfully wi

In [ ]:
!pip install torch

# 1.a IMAGE DATA PREPROCESSING :

In [ ]:
import cv2
import os
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.models as models

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

def resize_image(image, size=(256, 256)):
    return cv2.resize(image, size)

def rotate_image(image, angle=90):
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, rotation_matrix, (width, height))

def blur_image(image, kernel_size=(3, 3)):
    return cv2.GaussianBlur(image, kernel_size, 0)

def adjust_image(image):
    alpha, beta = 1.45, 20
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted_image

def flip_image(image):
    flip_dir = np.random.choice([0, 1])
    flipped_image = cv2.flip(image, flip_dir)
    return flipped_image

def grayscale_image(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def transform_and_save_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    output_folder="preprocessed_images"
    os.makedirs(output_folder, exist_ok=True)
    # Apply transformations
    resized_image = resize_image(image)
    rotated_image = rotate_image(resized_image)
    blurred_image = blur_image(resized_image)
    adjusted_image = adjust_image(resized_image)
    flipped_image = flip_image(resized_image)
    grayscaled_image = grayscale_image(resized_image)

    # Get the base filename without extension and file extension
    base_filename = os.path.splitext(os.path.basename(image_path))[0]
    file_extension = os.path.splitext(image_path)[1]

    # Save the transformed images with new naming convention
    cv2.imwrite(os.path.join(output_folder, f"{base_filename}_resized{file_extension}"), resized_image)
    cv2.imwrite(os.path.join(output_folder, f"{base_filename}_rotated{file_extension}"), rotated_image)
    cv2.imwrite(os.path.join(output_folder, f"{base_filename}_blurred{file_extension}"), blurred_image)
    cv2.imwrite(os.path.join(output_folder, f"{base_filename}_adjusted{file_extension}"), adjusted_image)
    cv2.imwrite(os.path.join(output_folder, f"{base_filename}_flipped{file_extension}"), flipped_image)
    cv2.imwrite(os.path.join(output_folder, f"{base_filename}_grayscale{file_extension}"), grayscaled_image)

def process_folder(folder_path):
    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        # Check if the file is an image
        if filename.lower().endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            transform_and_save_image(image_path)
            print(f"Processed and saved transformations for: {filename}")

# Example usage
folder_path = 'Images'
process_folder(folder_path)

Processed and saved transformations for: 1586_2712_image.jpg
Processed and saved transformations for: 540_2730_image.jpg
Processed and saved transformations for: 600_1436_image.jpg
Processed and saved transformations for: 247_3481_image.jpg
Processed and saved transformations for: 284_493_image.jpg
Processed and saved transformations for: 889_1061_image.jpg
Processed and saved transformations for: 1416_2501_image.jpg
Processed and saved transformations for: 191_279_image.jpg
Processed and saved transformations for: 1218_2234_image.jpg
Processed and saved transformations for: 568_3636_image.jpg
Processed and saved transformations for: 1372_3073_image.jpg
Processed and saved transformations for: 1013_3531_image.jpg
Processed and saved transformations for: 1042_2749_image.jpg
Processed and saved transformations for: 325_3237_image.jpg
Processed and saved transformations for: 1161_1168_image.jpg
Processed and saved transformations for: 951_2800_image.jpg
Processed and saved transformations

# **1.b IMAGE FEATURE EXTRACTION :**

In [ ]:
import os
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model

model = VGG16()
model=Model(inputs=model.inputs,outputs=model.layers[-2].output)

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

features={}
directory="preprocessed_images/"
for image in os.listdir(directory):
  image_path=directory+image
  img=load_img(image_path,target_size=(224,224))
  img=img_to_array(img)
  img=img.reshape((1,img.shape[0], img.shape[1], img.shape[2]))
  img=preprocess_input(img)
  feature=model.predict(img,verbose=0)
  features[image_path]=feature

for image_path, feature in features.items():
    features[image_path] = (feature - np.min(feature)) / (np.max(feature) - np.min(feature))

553467096/553467096 [==============================] - 7s 0us/step


In [ ]:
import pickle
pickle_file_path = '/content/drive/MyDrive/features_images2.pickle'
with open(pickle_file_path, 'wb') as pickle_file:
    pickle.dump(features, pickle_file)

print("Pickle file saved successfully.")


Pickle file saved successfully.


### **2.a TEXT DATA PREPROCESSING :**

In [ ]:
import re
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_sm",  disable=['ner'])
import json
from bs4 import BeautifulSoup
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
from nltk.tokenize import MWETokenizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
# Download NLTK resources
nltk.download('wordnet')

# Initialize WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
# Initialize MWETokenizer
tokenizer = MWETokenizer()


contractions_dict = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I had",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "iit will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

word_mapping = {
    'certainly': 'certain',
    'absolutely': 'absolute',
    'initially': 'initial',
    'really': 'real',
    'realy':'real',
    'perfectly':'perfect',
    'nicely' : 'nice',
    'frequently':'frequent',
    'fully':'full',
    'mainly':'main',
    'normally':'normal',
    'solidly':'solid',
    'highly':'high',
    'exactly':'exact',
    'totally':'total' ,
    'flawlessly':'flawless',
    'recently':'recent' ,
    'specially':'special',
    'surely':'sure',
    'previously':'previous',
    'aesthetically':'aesthetic',
    'especially':'especial',
    'mostly':'most',
    'definitely':'definite',
    'tighly':'tigh',
    'beautifully':'beautifull',
    'sadly':'sad',
    'thankfully':'thankful',
    'sorely':'sore',
    'extremely':'extreme',
    'directly':'direct',
    'smoothly':'smooth',
    'originally':'original',
    'securely':'secure',
    'additionally':'additional',
    'accidentally':'accidental',
    'probably':'probable',
    'overly':'over',
    'likely':'like',
    'barely':'bare',
    'slightly':'slight',
    'quickly':'quick',
    'brightly':'bright',
    'wonderfully':'wonderfull',
    'completely':'complete',
    'basically':'basic',
    'seriously':'serious',
    'actually':'actual',
    'obviously':'obvious',
    'eventually':'eventual',
    'easily':'easy',
    'slowly':'slow',
    'personally':'personal',
    'impressively':'impressive',
    'simply':'simple',
    'timely':'time',
    'readily':'ready',
    'consistently':'consistent',
    'pleasantly':'pleasant',
    'honestly':'honest',
    'poorly':'poor',
    'effectively':'effective',
    'sturdily':'sturd',
    'comfortably':'comfortable',
    'currently':'current',
    'constantly':'constant',
    'properly':'proper',
    'virtually':'virtual',
    'finally':'final',
    'correctly':'correct',
    'manually':'manual',
    'fairly':'fair'

  # Add more mappings as needed
}

def expand_contractions(text):
  contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())),
                                    flags=re.IGNORECASE | re.DOTALL)
  def expand_match(contraction):
      match = contraction.group(0)
      first_char = match[0]
      expanded_contraction = contractions_dict.get(match) \
          if contractions_dict.get(match) \
          else contractions_dict.get(match.lower())
      expanded_contraction = expanded_contraction
      return expanded_contraction
  text=str(text).strip()
  expanded_text = contractions_pattern.sub(expand_match, text)
  expanded_text = re.sub("'", "", expanded_text)
  return expanded_text

def create_strings_with_overlap(paragraph, overlap=4, words_per_string=7):
    words = paragraph.split()
    result_strings = []
    for i in range(0, len(words) - words_per_string + 1, words_per_string - overlap):
        result_strings.append(' '.join(words[i:i+words_per_string]))
    # Include the remaining words at the end
    remaining_words = words[-(words_per_string-overlap):]
    if remaining_words:
        result_strings.append(' '.join(remaining_words))
    return result_strings

def remove_abbreviations(text):
  #print("abbr before: ",text)
  for key, value in abbr.items():
    pattern = re.compile(key)
    text = pattern.sub(value, text)
  #print("abbr AFTER: ",text)
  return text

def remove_abbreviations1(text):
    pattern = re.compile(r'A[.:>]')
    text = pattern.sub("Answer", text)
    pattern2 = re.compile(r'Q[.:>]')
    text = pattern2.sub("Question", text)
    return text

def tokenizeSentence(text):
  return sent_tokenize(text)

def remove_special_chars(text):
  #print("sp before: ",text)
  pattern = r'[^A-Za-z0-9.\s]'
  cleaned_text = re.sub(pattern, '', text)
  #print("sp after: ",cleaned_text)
  return cleaned_text

def remove_periods(sentence):
  #print("rp before: ",sentence)
  cleaned_sentence = re.sub(r'\s*\.\s*$', '', sentence)
  cleaned_sentence = re.sub(r'(\w)\s*\.\s*(\w)', r'\1 \2', cleaned_sentence)
  #print("rp after: ",cleaned_sentence)
  return cleaned_sentence

def remove_period2(sentence):
  #print("rp2 before: ",sentence)
  cleaned_sentence = re.sub(r'[.]', ' ', sentence)
  #print("rp2 after: ",cleaned_sentence)
  return cleaned_sentence

def remove_stopwords_custom(text):
  pattern = re.compile(r'\b(?:' + '|'.join(word_list) + r')\b', flags=re.IGNORECASE)
  result_text = pattern.sub('', text)
  return result_text

def remove_stop_words_spacy(text):
  #print("stopword before: ",text)
  doc = nlp(text)
  cleaned_text = ' '.join(token.text for token in doc if not token.is_stop)
  #print("stopword after: ",cleaned_text)
  return cleaned_text

def lemmatize_text(text):
  doc = nlp(text)
  lemmatized_text = ' '.join(token.lemma_ for token in doc)
  return lemmatized_text

def remove_multiple_spaces(text):
  #print("multispace before: ",text)
  cleaned_string = re.sub(r'\s+', ' ', text)
  return cleaned_string

def preprocess_text(text):
  #text=remove_abbreviations1(text)
  text=str(text).strip()
  soup = BeautifulSoup(text, "html.parser")
  text = soup.get_text()
  text=expand_contractions(text)
  text=remove_special_chars(text)
  text=remove_periods(text)
  text=remove_period2(text)
  text=remove_multiple_spaces(text).strip()
  text=remove_stop_words_spacy(text)
  text=lemmatize_text(text)
  text=str(text).strip().lower()
  return preprocess_text_filter(text)

def lemmatize_word_filter(word):
    # Check if the word exists in the custom dictionary
    if word in word_mapping:
        return word_mapping[word]

    # Get the POS tag using the WordNetLemmatizer
    pos_tag = nltk.pos_tag([word])[0][1][0].upper()

    # Map POS tag to WordNet POS tag
    pos_dict = {'J': wordnet.ADJ, 'N': wordnet.NOUN, 'V': wordnet.VERB, 'R': wordnet.ADV}
    wordnet_pos = pos_dict.get(pos_tag, wordnet.NOUN)  # Default to noun if not found

    # Lemmatize the word with the appropriate POS tag
    lemma = lemmatizer.lemmatize(word, pos=wordnet_pos)
    return lemma

def preprocess_text_filter(text):

    # Parse the HTML content using BeautifulSoup
    text = str(text).strip()
    # Tokenization using MWETokenizer
    tokens = tokenizer.tokenize(text.split())
    tokens = [token for token in tokens if len(token) > 1]
    lemmatized_tokens = [lemmatize_word_filter(token) for token in tokens]

    return lemmatized_tokens

# preprocessed_data = {}

# for index, row in df.iterrows():
#     document_id = row['id']
#     image_metadata = row['Review Text']
#     # Preprocess image metadata
#     print(document_id)
#     preprocessed_metadata = preprocess_text(image_metadata)
#     preprocessed_data[document_id] = preprocessed_metadata
#     print(preprocessed_metadata)

# with open('preprocessed_data.json', 'w') as json_file:
#       json.dump(preprocessed_data, json_file)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **2.b TERM FREQUENCY-INVERSE DOCUMENT FREQUENCY SCORES FOR TEXT REVIEWS :**

In [ ]:
import pandas as pd
from collections import defaultdict
import math
import pandas as pd
from collections import defaultdict
import math


# Assuming df is your DataFrame
df = pd.read_csv('A2_Data.csv')

preprocessed_data = {}

for index, row in df.iterrows():
    document_id = row['id']
    image_metadata = row['Review Text']
    # Preprocess image metadata
    preprocessed_metadata = preprocess_text(image_metadata)
    preprocessed_data[document_id] = preprocessed_metadata

with open('preprocessed_data.json', 'w') as json_file:
      json.dump(preprocessed_data, json_file)
# Calculate TF for each term in each document
term_frequency = defaultdict(dict)

for doc_id, text in preprocessed_data.items():
    for term in text:
        if term not in term_frequency[doc_id]:
            term_frequency[doc_id][term] = 0
        term_frequency[doc_id][term] += 1

# Calculate IDF for each term
document_frequency = defaultdict(int)

vocabulary = set()

# Iterate over the preprocessed data and extract unique terms
for document_id, preprocessed_metadata in preprocessed_data.items():
    # Update the vocabulary with the terms in each document
    vocabulary.update(preprocessed_metadata)

# Convert the set of unique terms to a sorted list
vocabulary_list = sorted(vocabulary)

for doc_id, text in preprocessed_data.items():
    unique_terms = set(text)
    for term in unique_terms:
        document_frequency[term] += 1

num_documents = len(preprocessed_data)
inverse_document_frequency = {term: math.log(num_documents / (document_frequency[term] + 1)) for term in document_frequency}

# Calculate TF-IDF
tfidf_scores = defaultdict(dict)

for doc_id, term_freqs in term_frequency.items():
    doc_length = sum(term_freqs.values())
    for term, freq in term_freqs.items():
        tfidf_scores[doc_id][term] = (freq / doc_length) * inverse_document_frequency[term]

# Print TF-IDF scores for each document
for doc_id, scores in tfidf_scores.items():
    print("Document ID:", doc_id)
    for term, score in scores.items():
        print(term, ":", score)
    print()

Streaming output truncated to the last 5000 lines.
tone : 0.05627820004129408
want : 0.04601751384295504
play : 0.0405065738943974
peavey : 0.1127301567350646
tnt : 0.13222570422174876
100 : 0.08120665586433662
amp : 0.059162146679511605
place : 0.1301109649903012
low : 0.12265547184029736
belly : 0.13222570422174876
string : 0.04755162647915385
jack : 0.0843258787194829
secure : 0.0707284327771495
velcro : 0.17095249025897755
industrial : 0.1127301567350646
strength : 0.10885097467561877
wire : 0.08798226716473098
hide : 0.1127301567350646
beneath : 0.13222570422174876
end : 0.06095114917335291
high : 0.04856983969569928
recommend : 0.050082518880459496
durable : 0.0784867968960412
go : 0.05099778239287845
buy : 0.035902116040506644
good : 0.03027570947854217
adaptor : 0.11747789186940949
money : 0.061711108377652465

Document ID: 2284
super : 0.4962525820709987
light : 0.3644352074703636
weight : 0.48305639348085366
flexible : 0.7887801311231779
great : 0.1605614423789142
live : 0.62

In [ ]:
vocabulary

{'denver',
 'groove',
 'silver',
 '31',
 'shiny',
 'subtle',
 'ne',
 '4guitar',
 'make',
 'paranoid',
 'til',
 'brightness',
 'sheetrock',
 'nirvana',
 'baritone',
 'wife',
 'variable',
 'ingenious',
 'short',
 'fuse',
 'rubber',
 'machine',
 'accubass',
 '114e',
 'iklip',
 'fog',
 'envelope',
 'grover',
 'example',
 'doublepedal',
 'dynamic',
 'drawer',
 '58',
 'company',
 'solution',
 'straplock',
 'residue',
 'beveling',
 'cost',
 'found',
 'schmidt',
 'unusable',
 'scream',
 'protective',
 'foldable',
 'newb',
 'crown',
 'memberreviewer',
 'mid',
 'crossover',
 '125',
 'warmth',
 'diamond',
 'research',
 'nasty',
 'stub',
 'stompbox',
 'underneath',
 'attend',
 'handy',
 'worth',
 'backr',
 'softside',
 'risk',
 'splitter',
 'copper',
 'alter',
 'replace',
 'opening',
 'source',
 'enclose',
 'beneath',
 'dozen',
 'thingand',
 'fellow',
 'lr',
 'clipon',
 'supervision',
 'quasidixieland',
 'shove',
 'exact',
 '05252017still',
 'awkwardness',
 'yeah',
 'customer',
 '09',
 '45',
 'tou

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **DUMPING IMAGE FEATURES AND TEXT TF-IDF AS PICKLE FILE :**

In [ ]:
# Save the features dictionary to a pickle file
import pickle
output_pickle_path = "/content/drive/MyDrive/text_idf.pkl"

with open(output_pickle_path , 'wb') as f:
    pickle.dump(tfidf_scores, f)

print("Image features extracted and saved to pickle file.")

In [3]:
import pickle
invertedIndexFile = open('/content/drive/MyDrive/features_images.pickle', 'rb')                #reading the inverted index file  from inverted_index_up_final2 file.
features = pickle.load(invertedIndexFile)
invertedIndexFile.close()

In [4]:
import pickle
tfidf_folder = open('text_idf.pkl', 'rb')                #reading the inverted index file  from inverted_index_up_final2 file.
loaded_tfidf = pickle.load(tfidf_folder)
tfidf_folder.close()



In [6]:
import pandas as pd
csv_file_path = '/content/drive/MyDrive/A2_Data.csv'
df = pd.read_csv(csv_file_path)


In [7]:
csv_file = '/content/drive/MyDrive/split_image_links.csv'
df3 = pd.read_csv(csv_file)


# **3.a  IMAGE BASED RETRIEVAL :**

In [12]:
import urllib.request
from PIL import Image
import os
import io
import cv2
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
import numpy as np
import re
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_sm",  disable=['ner'])
import json
from bs4 import BeautifulSoup
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
from nltk.tokenize import MWETokenizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
# Download NLTK resources
nltk.download('wordnet')

# Initialize WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
# Initialize MWETokenizer
tokenizer = MWETokenizer()

def resize_image(image_path, size=(256, 256)):
    image = cv2.imread(image_path)
    return cv2.resize(image, size)

# Function to display image from URL
def display_image_from_url(image_url):
    with urllib.request.urlopen(image_url) as url:
        image_data = url.read()
    image = Image.open(io.BytesIO(image_data))
    return image


# Function to save image to directory
def save_image(image, directory, filename):
    if not os.path.exists(directory):
        os.makedirs(directory)
    image.save(os.path.join(directory, filename))

# Taking input image URL from the user
image_url = input("Enter the image URL: ")

# Displaying the image
image = display_image_from_url(image_url)


# Taking description from the user
description = input("Enter a description for the image: ")

# Saving the image to a directory
directory = "input_image"
filename = "image.jpg"  # You can change the filename as needed
output_folder="preprocessed_input_image"
os.makedirs(output_folder, exist_ok=True)
save_image(image, directory, filename)
resized_image=resize_image("input_image/image.jpg")
cv2.imwrite(os.path.join(output_folder, "image_resized.jpg"), resized_image)

model = VGG16()
model=Model(inputs=model.inputs,outputs=model.layers[-2].output)
features_input={}
image_path="preprocessed_input_image/image_resized.jpg"
img=load_img(image_path,target_size=(224,224))
img=img_to_array(img)
img=img.reshape((1,img.shape[0], img.shape[1], img.shape[2]))
img=preprocess_input(img)
feature_input=model.predict(img,verbose=0)
features_input[image_path]=feature_input
for image_path, feature in features_input.items():
    features_input[image_path] = (feature - np.min(feature)) / (np.max(feature) - np.min(feature))

import numpy as np
#from sklearn.metrics.pairwise import cosine_similarity
import warnings
import pandas as pd
import math
from collections import Counter

# Suppress all warnings
warnings.filterwarnings("ignore")



contractions_dict = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I had",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "iit will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

word_mapping = {
    'certainly': 'certain',
    'absolutely': 'absolute',
    'initially': 'initial',
    'really': 'real',
    'realy':'real',
    'perfectly':'perfect',
    'nicely' : 'nice',
    'frequently':'frequent',
    'fully':'full',
    'mainly':'main',
    'normally':'normal',
    'solidly':'solid',
    'highly':'high',
    'exactly':'exact',
    'totally':'total' ,
    'flawlessly':'flawless',
    'recently':'recent' ,
    'specially':'special',
    'surely':'sure',
    'previously':'previous',
    'aesthetically':'aesthetic',
    'especially':'especial',
    'mostly':'most',
    'definitely':'definite',
    'tighly':'tigh',
    'beautifully':'beautifull',
    'sadly':'sad',
    'thankfully':'thankful',
    'sorely':'sore',
    'extremely':'extreme',
    'directly':'direct',
    'smoothly':'smooth',
    'originally':'original',
    'securely':'secure',
    'additionally':'additional',
    'accidentally':'accidental',
    'probably':'probable',
    'overly':'over',
    'likely':'like',
    'barely':'bare',
    'slightly':'slight',
    'quickly':'quick',
    'brightly':'bright',
    'wonderfully':'wonderfull',
    'completely':'complete',
    'basically':'basic',
    'seriously':'serious',
    'actually':'actual',
    'obviously':'obvious',
    'eventually':'eventual',
    'easily':'easy',
    'slowly':'slow',
    'personally':'personal',
    'impressively':'impressive',
    'simply':'simple',
    'timely':'time',
    'readily':'ready',
    'consistently':'consistent',
    'pleasantly':'pleasant',
    'honestly':'honest',
    'poorly':'poor',
    'effectively':'effective',
    'sturdily':'sturd',
    'comfortably':'comfortable',
    'currently':'current',
    'constantly':'constant',
    'properly':'proper',
    'virtually':'virtual',
    'finally':'final',
    'correctly':'correct',
    'manually':'manual',
    'fairly':'fair'

  # Add more mappings as needed
}

def expand_contractions(text):
  contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())),
                                    flags=re.IGNORECASE | re.DOTALL)
  def expand_match(contraction):
      match = contraction.group(0)
      first_char = match[0]
      expanded_contraction = contractions_dict.get(match) \
          if contractions_dict.get(match) \
          else contractions_dict.get(match.lower())
      expanded_contraction = expanded_contraction
      return expanded_contraction
  text=str(text).strip()
  expanded_text = contractions_pattern.sub(expand_match, text)
  expanded_text = re.sub("'", "", expanded_text)
  return expanded_text

def create_strings_with_overlap(paragraph, overlap=4, words_per_string=7):
    words = paragraph.split()
    result_strings = []
    for i in range(0, len(words) - words_per_string + 1, words_per_string - overlap):
        result_strings.append(' '.join(words[i:i+words_per_string]))
    # Include the remaining words at the end
    remaining_words = words[-(words_per_string-overlap):]
    if remaining_words:
        result_strings.append(' '.join(remaining_words))
    return result_strings

def remove_abbreviations(text):
  #print("abbr before: ",text)
  for key, value in abbr.items():
    pattern = re.compile(key)
    text = pattern.sub(value, text)
  #print("abbr AFTER: ",text)
  return text

def remove_abbreviations1(text):
    pattern = re.compile(r'A[.:>]')
    text = pattern.sub("Answer", text)
    pattern2 = re.compile(r'Q[.:>]')
    text = pattern2.sub("Question", text)
    return text

def tokenizeSentence(text):
  return sent_tokenize(text)

def remove_special_chars(text):
  #print("sp before: ",text)
  pattern = r'[^A-Za-z0-9.\s]'
  cleaned_text = re.sub(pattern, '', text)
  #print("sp after: ",cleaned_text)
  return cleaned_text

def remove_periods(sentence):
  #print("rp before: ",sentence)
  cleaned_sentence = re.sub(r'\s*\.\s*$', '', sentence)
  cleaned_sentence = re.sub(r'(\w)\s*\.\s*(\w)', r'\1 \2', cleaned_sentence)
  #print("rp after: ",cleaned_sentence)
  return cleaned_sentence

def remove_period2(sentence):
  #print("rp2 before: ",sentence)
  cleaned_sentence = re.sub(r'[.]', ' ', sentence)
  #print("rp2 after: ",cleaned_sentence)
  return cleaned_sentence

def remove_stopwords_custom(text):
  pattern = re.compile(r'\b(?:' + '|'.join(word_list) + r')\b', flags=re.IGNORECASE)
  result_text = pattern.sub('', text)
  return result_text

def remove_stop_words_spacy(text):
  #print("stopword before: ",text)
  doc = nlp(text)
  cleaned_text = ' '.join(token.text for token in doc if not token.is_stop)
  #print("stopword after: ",cleaned_text)
  return cleaned_text

def lemmatize_text(text):
  doc = nlp(text)
  lemmatized_text = ' '.join(token.lemma_ for token in doc)
  return lemmatized_text

def remove_multiple_spaces(text):
  #print("multispace before: ",text)
  cleaned_string = re.sub(r'\s+', ' ', text)
  return cleaned_string

def preprocess_text(text):
  #text=remove_abbreviations1(text)
  text=str(text).strip()
  soup = BeautifulSoup(text, "html.parser")
  text = soup.get_text()
  text=expand_contractions(text)
  text=remove_special_chars(text)
  text=remove_periods(text)
  text=remove_period2(text)
  text=remove_multiple_spaces(text).strip()
  text=remove_stop_words_spacy(text)
  text=lemmatize_text(text)
  text=str(text).strip().lower()
  return preprocess_text_filter(text)

def lemmatize_word_filter(word):
    # Check if the word exists in the custom dictionary
    if word in word_mapping:
        return word_mapping[word]

    # Get the POS tag using the WordNetLemmatizer
    pos_tag = nltk.pos_tag([word])[0][1][0].upper()

    # Map POS tag to WordNet POS tag
    pos_dict = {'J': wordnet.ADJ, 'N': wordnet.NOUN, 'V': wordnet.VERB, 'R': wordnet.ADV}
    wordnet_pos = pos_dict.get(pos_tag, wordnet.NOUN)  # Default to noun if not found

    # Lemmatize the word with the appropriate POS tag
    lemma = lemmatizer.lemmatize(word, pos=wordnet_pos)
    return lemma

def preprocess_text_filter(text):

    # Parse the HTML content using BeautifulSoup
    text = str(text).strip()
    # Tokenization using MWETokenizer
    tokens = tokenizer.tokenize(text.split())
    tokens = [token for token in tokens if len(token) > 1]
    lemmatized_tokens = [lemmatize_word_filter(token) for token in tokens]

    return lemmatized_tokens
def preprocess_data():
    # df = pd.read_csv('A2_Data.csv')
    preprocessed_data = {}

    for index, row in df.iterrows():
        document_id = row['id']
        image_metadata = row['Review Text']
        preprocessed_metadata = preprocess_text(image_metadata)
        preprocessed_data[document_id] = preprocessed_metadata
    return preprocessed_data

def preprocess_input2(text):
    return preprocess_text(text)

def calculate_tf(document):
    tf = {}
    word_count = len(document)
    word_freq = Counter(document)
    for word, freq in word_freq.items():
        tf[word] = freq / word_count
    return tf

def calculate_idf(documents):
    idf = {}
    total_docs = len(documents)
    for doc in documents.values():
        for term in set(doc):
            if term not in idf:
                doc_count = sum(1 for d in documents.values() if term in d)
                idf[term] = math.log(total_docs / (1 + doc_count))
    return idf

def cosine_similarity_doc(doc1, doc2):
    dot_product = sum(doc1.get(term, 0) * doc2.get(term, 0) for term in set(doc1) & set(doc2))
    magnitude_doc1 = math.sqrt(sum(value ** 2 for value in doc1.values()))
    magnitude_doc2 = math.sqrt(sum(value ** 2 for value in doc2.values()))
    return dot_product / (magnitude_doc1 * magnitude_doc2)


# Assuming df is your DataFrame
# df = pd.read_csv('A2_Data.csv')


preprocessed_data = {}

for index, row in df.iterrows():
    document_id = row['id']
    image_metadata = row['Review Text']
    preprocessed_metadata = preprocess_text(image_metadata)
    preprocessed_data[document_id] = preprocessed_metadata
def cosine_similarity_d(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_vec1 = np.linalg.norm(v1)
    norm_vec2 = np.linalg.norm(v2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

def calculate(preprocessed_data) :
    # preprocessed_data = preprocess_data()
    input_text = description
    input_terms = preprocess_input2(input_text)
    preprocessed_data["input_image/image.jpg"] = input_terms
    tf_documents = {doc_id: calculate_tf(doc) for doc_id, doc in preprocessed_data.items()}
    idf = calculate_idf(preprocessed_data)
    tfidf_documents = {}
    for doc_id, tf_doc in tf_documents.items():
        tfidf_doc = {}
        for term, tf in tf_doc.items():
            tfidf_doc[term] = tf * idf.get(term, 0)
            tfidf_documents[doc_id] = tfidf_doc
    return tfidf_documents

    # Feature vector of the input image
    # Convert feature vectors to lists
image_input = features_input['preprocessed_input_image/image_resized.jpg'].tolist()

# Calculate cosine similarity for each comparison
similarities = {}
for image_path, feature_vector in features.items():
    # Convert feature vectors to NumPy arrays
    image_vector_list = feature_vector.tolist()


   # Calculate cosine similarity
    similarity = cosine_similarity_d(image_input[0], image_vector_list[0])

    similarities[image_path] = similarity

# Sort the similarities dictionary by cosine similarity in descending order
sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

ids = []
scores = []
index=[]
# Print the top 3 similarities
print("Top 3 similarity scores:")
for i, (image_path, similarity) in enumerate(sorted_similarities[:15], start=1):
    numeric_part = image_path.split('/')[1].split('_')[1]
    index_part = image_path.split('/')[1].split('_')[0]
    if numeric_part not in ids:
        ids.append(numeric_part)
        scores.append(similarity)
        index.append(index_part)

# unique_numeric_parts[:3]
ids=ids[:3]
scores=scores[:3]
index=index[:3]


#df3=pd.read_csv('split_image_links.csv')

# # Filter rows based on the IDs
filtered_df = df[df['id'].astype(str).isin(ids)]
image_urls = []
for i in index:
    # Assuming 'i' is the index position
    image_url = df3.iloc[int(i)-1]['image_link'] # Extract image URL from the ith row
    image_urls.append(image_url)


tfidf_documents= calculate(preprocessed_data)

integer_list = [int(item) for item in ids]
document_ids_set =integer_list

# Calculate cosine similarity with the input text for each document ID in the set
cosine_similarities_set = {}
for doc_id in document_ids_set:
    similarity = cosine_similarity_doc(tfidf_documents["input_image/image.jpg"], tfidf_documents[doc_id])
    cosine_similarities_set[doc_id] = similarity

composite_similarities = []
composite_similarities_dict = {}
similarity_values = list(cosine_similarities_set.values())

image_data = []
image_data_composite=[]

print("-------------Ranked Images based on Consine  Similarity of input image-----------------")
for i in range(3):
    composite_similarity = (scores[i] + similarity_values[i]) / 2
    composite_similarities.append(composite_similarity)
    filtered_row = filtered_df[filtered_df['id'] == int(ids[i])]
    composite_similarities_dict[ids[i]] = composite_similarity

    #image_url = filtered_row['Image'].iloc[0]
    image_review = filtered_row['Review Text'].iloc[0]
    image_dict = {
        "Image ID": ids[i],
        "Image URL": image_urls[i],
        "Review": image_review,
        "Cosine Similarity of Images": scores[i],
        "Cosine Similarity of Text": similarity_values[i],
        "Composite Similarity": composite_similarity
    }
    image_data.append(image_dict)
    print("Image ID: ", ids[i])
    print("Image URL: ", image_urls[i])
    print("Review: ", image_review)
    print("Cosine similarity of images - ", scores[i])
    print("Cosine similarity of text - ", similarity_values[i])
    print("Composite similarity - ", composite_similarity)
    print()
    print()


sorted_image_data = sorted(image_data, key=lambda x: x["Composite Similarity"], reverse=True)

print("-------------Ranked Images based on Composite Similarity-------------")

for image_info in sorted_image_data:
    image_dct = {
        "Image ID": image_info["Image ID"],
        "Image URL": image_info["Image URL"],
        "Review": image_info["Review"],
        "Cosine Similarity of Images": image_info["Cosine Similarity of Images"],
        "Cosine Similarity of Text": image_info["Cosine Similarity of Text"],
        "Composite Similarity": image_info["Composite Similarity"]
    }
    image_data_composite.append(image_dct)
    print("Image ID: ", image_info["Image ID"])
    print("Image URL: ", image_info["Image URL"])
    print("Review: ", image_info["Review"])
    print("Cosine Similarity of Images: ", image_info["Cosine Similarity of Images"])
    print("Cosine Similarity of Text: ", image_info["Cosine Similarity of Text"])
    print("Composite Similarity: ", image_info["Composite Similarity"])
    print()


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Enter the image URL: https://images-na.ssl-images-amazon.com/images/I/51zHQdv4n1L._SY88.jpg
Enter a description for the image: This is a fine EQ.  It sounds good and I'm  thinking of getting another one  for right and left  speaker.
Top 3 similarity scores:
-------------Ranked Images based on Consine  Similarity of input image-----------------
Image ID:  672
Image URL:  https://images-na.ssl-images-amazon.com/images/I/51zHQdv4n1L._SY88.jpg
Review:  This is a fine EQ.  It sounds good and I'm  thinking of getting another one  for right and left  speaker.
Cosine similarity of images -  0.9999860388633867
Cosine similarity of text -  1.0000000000000002
Composite similarity -  0.9999930194316935


Image ID:  2738
Image URL:  https://images-na.ssl-images-amazon.com/images/I/716NnwQnfrL._SY88.jpg
Review:  Kit is awesome. I play in my garage just for personal enjoyment not for performances or anything. Once you take the time to break down all the settings, your able to dial in pretty much any 

In [16]:
with open('image_data.pkl', 'wb') as f:
    pickle.dump(image_data, f)

In [17]:
with open('image_data.pkl', 'rb') as f:
    image_data = pickle.load(f)

In [19]:
with open('image_data_composite.pkl', 'wb') as f:
    pickle.dump(image_data_composite, f)

In [18]:
with open('image_data_composite.pkl', 'rb') as f:
    image_data_composite = pickle.load(f)

# **3.b TEXT BASED RETRIEVAL :**

In [21]:
import math
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Input document provided by the user
input_text = description

# Preprocess the input text
def preprocess_input2(text):
    return preprocess_text(text)

input_terms = preprocess_input2(input_text)

# Combine input_terms with existing documents
preprocessed_data["input_image/image.jpg"] = input_terms

# Calculate TF (Term Frequency) for each term in each document
def calculate_tf(document):
    tf = {}
    word_count = len(document)
    word_freq = Counter(document)
    for word, freq in word_freq.items():
        tf[word] = freq / word_count
    return tf

tf_documents = {doc_id: calculate_tf(doc) for doc_id, doc in preprocessed_data.items()}

# Calculate IDF (Inverse Document Frequency) for each term
def calculate_idf(documents):
    idf = {}
    total_docs = len(documents)
    for doc in documents.values():
        for term in set(doc):
            if term not in idf:
                doc_count = sum(1 for d in documents.values() if term in d)
                idf[term] = math.log(total_docs / (1 + doc_count))
    return idf

idf = calculate_idf(preprocessed_data)

# Calculate TF-IDF for each term in each document
tfidf_documents = {}
for doc_id, tf_doc in tf_documents.items():
    tfidf_doc = {}
    for term, tf in tf_doc.items():
        tfidf_doc[term] = tf * idf.get(term, 0)
    tfidf_documents[doc_id] = tfidf_doc


# Compute cosine similarity with the input document for each document
cosine_similarities = {
    doc_id: cosine_similarity_doc(tfidf_documents["input_image/image.jpg"], tfidf_doc)
    for doc_id, tfidf_doc in tfidf_documents.items() if doc_id != "input_image/image.jpg"
}


sorted_cosine_similarities = sorted(cosine_similarities.items(), key=lambda x: x[1], reverse=True)

top_3_similarities = sorted_cosine_similarities[:3]

top_document_ids = set()

for doc_id, _ in top_3_similarities:
    top_document_ids.add(doc_id)


def output_formating(final_result_text,image_ids_to_extract) :
    # Filter rows based on image IDs
    filtered_rows = df[df['id'].isin(image_ids_to_extract)]
    # Extract review text for the filtered rows
    review_texts = filtered_rows['Review Text']
    review_data = []

    for _, row in filtered_rows.iterrows():
       id_val = row['id']
       review_text_val = row['Review Text']
       review_data.append({"id": id_val, "review_text": review_text_val})
    merged_review_data = []
    for entry1 in final_result_text:
        for entry2 in review_data:
            if entry1['id'] == entry2['id']:
                merged_entry = {**entry1, **entry2}
                merged_review_data.append(merged_entry)
                break
    key_order = ['id', 'image_urls', 'review_text', 'cosine_similarity_score', 'cosine_similarities_images', 'composite_similarity']
    sorted_data = [{k: entry[k] for k in key_order} for entry in merged_review_data]
    return sorted_data

def cosine_similarity_matching(features_input,features_text_image):
    image_input = features_input['preprocessed_input_image/image_resized.jpg'].tolist()


    # Calculate cosine similarity for each comparison
    similarities2 = {}
    for image_path, feature_vector in features_text_image.items():
        # Convert feature vectors to NumPy arrays
        image_vector_list = feature_vector.tolist()

         # Calculate cosine similarity
        similarity = cosine_similarity_d(image_input[0], image_vector_list[0])

        similarities2[image_path] = similarity

        # Sort the similarities dictionary by cosine similarity in descending order
        sorted_similarities2 = sorted(similarities2.items(), key=lambda x: x[1], reverse=True)
    return similarities2


# Function to extract features from an image
def extract_features(image_path, model):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    img = preprocess_input(img)
    feature = model.predict(img, verbose=0)
    feature_normalized = (feature - np.min(feature)) / (np.max(feature) - np.min(feature))
    return feature_normalized

def vgg16_model() :
    # Define the model
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

    # Directory containing images
    image_folder = "text_retrived_image"

    # Dictionary to store features
    features_text_image = {}

    # Extract features for each image in the folder
    for root, dirs, files in os.walk(image_folder):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.png'):  # Adjust file formats as needed
               image_path = os.path.join(root, file)
               feature = extract_features(image_path, model)
               features_text_image[os.path.join(root.split('/')[-1], file)] = feature
    for image_path, feature in features_text_image.items():
        features_text_image[image_path] = (feature - np.min(feature)) / (np.max(feature) - np.min(feature))

    return features_text_image

# Function to display an image from a URL
def display_image_from_url(image_url):
    with urllib.request.urlopen(image_url) as url:
        image_data = url.read()
    image = Image.open(io.BytesIO(image_data))
    image.show()  # Displaying the image
    return image

# Function to save an image to a directory
def save_image(image, directory, filename):
    if not os.path.exists(directory):
        os.makedirs(directory)
    image.save(os.path.join(directory, filename))

def resize_image(image, size=(256, 256)):
    return image.resize(size)

def get_image_from_links(image_links) :
   # Dictionary of image URLs
   image_urls = image_links

   # Directory to save images
   output_folder = "text_retrived_image"

   for filename, url in image_urls.items():
       image = display_image_from_url(url)  # Display the image
       resized_image = resize_image(image, size=(256, 256))  # Resize the image
       save_image(resized_image, output_folder, filename)   # Save the image to the directory

# Convert to JSON format
def save_in_format(top_3_similarities):
    json_data = {"top_3_similarities": []}
    for doc_id, similarity in top_3_similarities:
        json_data["top_3_similarities"].append({
        "id": doc_id,
        "cosine_similarity_score": similarity
    })
    return json_data

def combined_data(json_data,formatted_image_links_dict) :
    combined_data = []

    for similarity_info in json_data['top_3_similarities']:
       image_id = similarity_info['id']
       similarity_score = similarity_info['cosine_similarity_score']

    # Check if image ID exists in the formatted image links dictionary
       image_key = f"{image_id}"
       if image_key in formatted_image_links_dict:
          image_urls = formatted_image_links_dict[image_key]
          combined_data.append({'id': image_id, 'cosine_similarity_score': similarity_score, 'image_urls': image_urls})

    return(combined_data)

def get_matched_image_links(top_document_ids):
    given_image_ids = top_document_ids

    # df = pd.read_csv('split_image_links.csv')

    # Extract rows from the DataFrame where the 'id' column matches with IDs in your set
    selected_rows = df3[df3['id'].isin(given_image_ids)]

    image_links_dict = {}
    image_links = {}


    # Iterate through selected rows and populate the dictionary
    for index, row in selected_rows.iterrows():
        index_id = row['id']
        image_link = row['image_link']
        if index_id in image_links_dict:
           image_links_dict[index_id].append(image_link)
        else:
           image_links_dict[index_id] = [image_link]
        image_links[f"{index}_{index_id}_image.jpg"] = image_link


# Combine URLs for all image IDs with multiple URLs into a single key
    for index_id, urls in image_links_dict.items():
        if len(urls) > 1:
            combined_urls = ' '.join(urls)
            image_links_dict[index_id] = [combined_urls]

# Format the dictionary keys as specified
    formatted_image_links_dict = {}
    for image_id, urls in image_links_dict.items():
        formatted_image_links_dict[f"{image_id}"] = urls
    return formatted_image_links_dict,image_links


# Define a function to extract the numeric part of the keys
def extract_numeric_part(key):
    # Use regular expression to find the numeric part
    numeric_part = re.search(r'\d+', key)
    if numeric_part:
        return int(numeric_part.group())
    return 0  # Return 0 if no numeric part is found

def output_postprocessing(sorted_similarities_cleaned,combined_data_format):
    # Sort the list based on the numeric part of the keys
     sorted_similarities_sorted = sorted(sorted_similarities_cleaned, key=lambda x: extract_numeric_part(x[0]))
     pattern = r'^\d+_|\_image\.jpg$'
     # Iterate through the data and apply the pattern to each key
     result = [(re.sub(pattern, '', key), float(value)) for key, value in sorted_similarities_sorted]
     unique_data = {}

     # Iterate through the data and populate the dictionary
     for key, value in result:
         if key in unique_data:
            unique_data[key].append(value)
         else:
            unique_data[key] = [value]

     unique_data_formatted = []

     # Iterate through the data and populate the formatted list
     for key, values in unique_data.items():
         formatted_entry = {'id': key, 'cosine_similarities_images': ', '.join(map(str, values))}
         unique_data_formatted.append(formatted_entry)

     # Convert data2 to a dictionary for easy merging
     data2_dict = {entry['id']: entry['cosine_similarities_images'] for entry in unique_data_formatted}

     # Merge data1 and data2 based on 'id'
     merged_data = []
     for entry in combined_data_format:
         merged_entry = entry.copy()
         if str(entry['id']) in data2_dict:
            merged_entry['cosine_similarities_images'] = data2_dict[str(entry['id'])]
         else:
            merged_entry['cosine_similarities_images'] = None
         merged_data.append(merged_entry)


     # Calculate the composite similarity
     for entry in merged_data:
         cosine_similarities_images = [float(value) for value in entry['cosine_similarities_images'].split(', ')]
         max_cosine_similarity_images = max(cosine_similarities_images, default=0)
         composite_similarity = (entry['cosine_similarity_score'] + max_cosine_similarity_images) / 2
         entry['composite_similarity'] = composite_similarity

     print("------------Retrival based on cosine similarity of input image Review Text-------------------")
     # Print the merged data along with composite similarity
     for entry in merged_data:
         print(f"id: {entry['id']}")
         print(f"image_urls: {', '.join(entry['image_urls'])}")
         print(f"cosine_similarity_score: {entry['cosine_similarity_score']}")
         print(f"cosine_similarities_images: {entry['cosine_similarities_images']}")
         print(f"composite_similarity: {entry['composite_similarity']}")
         print()
         print()


     sorted_merged_data = sorted(merged_data, key=lambda x: x['composite_similarity'], reverse=True)


     print("------------Retrival based on composite similarity-------------------")
     # Print the sorted data
     for entry in sorted_merged_data:
         print(f"id: {entry['id']}")
         print(f"image_urls: {', '.join(entry['image_urls'])}")
         print(f"cosine_similarity_score: {entry['cosine_similarity_score']}")
         print(f"cosine_similarities_images: {entry['cosine_similarities_images']}")
         print(f"composite_similarity: {entry['composite_similarity']}")
         print()
         print()

     return sorted_merged_data

json_data=save_in_format(top_3_similarities)


formatted_image_links_dict,image_links=get_matched_image_links(top_document_ids)

get_image_from_links(image_links)

combined_data_format=combined_data(json_data,formatted_image_links_dict)

features_text_image=vgg16_model()
similarities_score=cosine_similarity_matching(features_input,features_text_image)

similarities_score
# Remove the prefix "text_retrived_image/" from the keys
sorted_similarities_cleaned = [(key.replace('text_retrived_image/', ''), value) for key, value in similarities_score.items()]
final_result_text=output_postprocessing(sorted_similarities_cleaned,combined_data_format)
final_output=output_formating(final_result_text,top_document_ids)


#Till now i got top doc_id and top3_similarity_scores

------------Retrival based on cosine similarity of input image Review Text-------------------
id: 672
image_urls: https://images-na.ssl-images-amazon.com/images/I/51fyLunjnYL._SY88.jpg https://images-na.ssl-images-amazon.com/images/I/51zHQdv4n1L._SY88.jpg
cosine_similarity_score: 1.0000000000000002
cosine_similarities_images: 0.861157667140689, 0.9717908437849351
composite_similarity: 0.9858954218924676


id: 3670
image_urls: https://images-na.ssl-images-amazon.com/images/I/71n5KZ8RHGL._SY88.jpg
cosine_similarity_score: 0.2117715511735518
cosine_similarities_images: 0.2078780777536238
composite_similarity: 0.2098248144635878


id: 3736
image_urls: https://images-na.ssl-images-amazon.com/images/I/61QWC8bH9qL._SY88.jpg
cosine_similarity_score: 0.19967129045840568
cosine_similarities_images: 0.22910733146252554
composite_similarity: 0.2143893109604656


------------Retrival based on composite similarity-------------------
id: 672
image_urls: https://images-na.ssl-images-amazon.com/images/

In [22]:
with open('final_output.pkl', 'wb') as f:
    pickle.dump(final_output, f)

In [23]:
with open('final_output.pkl', 'rb') as f:
    final_output = pickle.load(f)

In [ ]:
with open('image_data_composite.pkl', 'rb') as f:
    image_data_composite = pickle.load(f)

# **5.Results and Analysis:**

In [24]:
print("---------Image based Retrival--------------")
print(image_data_composite)
print()
print()
print("---------Text based Retrival--------------")
print(final_output)

---------Image based Retrival--------------
[{'Image ID': '672', 'Image URL': 'https://images-na.ssl-images-amazon.com/images/I/51zHQdv4n1L._SY88.jpg', 'Review': "This is a fine EQ.  It sounds good and I'm  thinking of getting another one  for right and left  speaker.", 'Cosine Similarity of Images': 0.9999860388633867, 'Cosine Similarity of Text': 1.0000000000000002, 'Composite Similarity': 0.9999930194316935}, {'Image ID': '2738', 'Image URL': 'https://images-na.ssl-images-amazon.com/images/I/716NnwQnfrL._SY88.jpg', 'Review': 'Kit is awesome. I play in my garage just for personal enjoyment not for performances or anything. Once you take the time to break down all the settings, your able to dial in pretty much any kit and sound. With the expansion options and the relatively inexpensive parts expanding is easy and fun.\n\nAfter a few weeks of daily use for at least an hour a day it still looks and plays beautifully. Overall one of the best purchases I could have made.', 'Cosine Similar

**Comparision of techniques :**

To compare which retrieval technique gives a better similarity score  two techniques: Image-based retrieval and Text-based retrieval. Each technique calculates a composite similarity score based on different aspects:

Image-based retrieval: Retrives top 3 images based cosine smilarity of images.Then fetched corresponding  3 review text and their cosine similarity with input text also calculated. Composite similarity is then calculated  using cosine similarity of images and Text is calculated individually .

Text-based retrieval: Retrives top 3 Review text based cosine smilarity of review texts .Then fetched corresponding  3 lists of image urls  and their cosine similarity with input image also calculated. Composite similarity is then calculated  using cosine similarity of Text and Image(max.cosine similarity) is calculated individually .

Now, let's analyze which technique gives a better similarity score:

**Image-based retrieval:**

Composite similarity: 0.9999 (for Image ID: 672)

Composite similarity: 0.3398 (for Image ID: 2738)

Composite similarity: 0.3210 (for Image ID: 1238)

**Text-based retrieval:**

Composite similarity: 0.9858 (for ID: 672)

Composite similarity: 0.2143 (for ID: 3736)

Composite similarity: 0.2098 (for ID: 3670)

From the data provided, we observe that the Image-based retrieval technique generally yields higher composite similarity scores compared to Text-based retrieval.

**Reason :**
The Image-based retrieval technique likely outperforms the Text-based retrieval technique due to:

Richness of representation: Images capture diverse visual features better than text.
Semantic gap bridging: Deep learning models extract meaningful features from images, addressing the semantic gap more effectively.
Subjectivity and ambiguity: Textual descriptions can vary widely, leading to challenges in measuring similarity.
Contextual understanding: Images offer a more holistic representation, aiding in contextual understanding.
Data quality and quantity: Larger, more diverse image datasets may result in better generalization and coverage.
These factors collectively contribute to the observed higher composite similarity scores for Image-based retrieval.

**Challenges:**
Choosing best pre-trained model for image feature extraction is difficult task becoz some pretrained models takes lot of time to give features and also give a large size feature of few GB's.
Dataset contains some urls that failed to download images.








